# Logistic Map Chaos (Sine and Tent maps also)
- Alex Kim 2023-10-26
- https://ajpkim.com/posts/bifurcation-diagrams.html
- I'm currently reading "Chaos" (Gleick 1987) and wanted to play with the ideas and recreate some of the images in the book
- Logitic Map Function (Robert May): https://en.wikipedia.org/wiki/Logistic_map
  - $x_{n+1}=rx_{n}(1-x_{n})$ 
 
## Bifurcation Diagram

- https://en.wikipedia.org/wiki/Bifurcation_diagram

"Consider a system of differential equations that describes some physical quantity, that for concreteness could represent one of three examples: 1. the position and velocity of an undamped and frictionless pendulum, 2. a neuron's membrane potential over time, and 3. the average concentration of a virus in a patient's bloodstream. The differential equations for these examples include *parameters* that may affect the output of the equations. Changing the pendulum's mass and length will affect its oscillation frequency, changing the magnitude of injected current into a neuron may transition the membrane potential from resting to spiking, and the long-term viral load in the bloodstream may decrease with carefully timed treatments.

In general, researchers may seek to quantify how the long-term (asymptotic) behavior of a system of differential equations changes if a parameter is changed. In the dynamical systems branch of mathematics, a bifurcation diagram quantifies these changes by showing how fixed points, periodic orbits, or chaotic attractors of a system change as a function of bifurcation parameter. Bifurcation diagrams are used to visualize these changes."

In [ ]:
from typing import List, Callable
import math

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
"""
Functions for creating bifurcation plots.

For some given recursive map function, like the logistic map, we want to record some number of 
state variable values once the function has reached 'settled behavior'. Doing this for each 
value in the control parameter space we're investigating allows us to visualize how the system's terminal 
behavior changes with respect to the control param. For example, we may see that for some control param 
val 'a' the terminal behavior has period 4 while for another val 'b' it has period 8.
"""

def logistic_map(x, r):
    return r * (x - x**2)


def sine_map(x, r):
    return r * math.sin(math.pi * x)
    

def tent_map(x, r):
    if x < 0.5:
        return r * x
    else:
        return r * (1 - x)


# Quadratic and cubic maps overflow
def quadratic_map(x, r):
    return r * x * (1 - x) + x


def cubic_map(x, r):
    return r * x * (1 - x**2)


def iterate_f(
    f: Callable,
    n: int,
    x_init: float,
    control_param: float,
    store_vals: bool = True,
    **kwargs
):
    """
    Recursively iterate given function and return either all intermediate vals or just the last val.
    """
    x = x_init
    vals = []
    for _ in range(n):
        x = f(x, control_param, **kwargs)
        if store_vals:
            vals.append(x)
    return vals if store_vals else x


def f_steady_vals(
    f: Callable,
    steady_n: int,
    transient_n: int,
    x_init: float,
    control_param: float,
    **kwargs
) -> List[float]:
    """
    Get 'steady_n' number of state variables after running 'transient_n' number of steps of
    given function for some constant control param value.
    """
    x = iterate_f(f, transient_n, x_init, control_param, store_vals=False, **kwargs)
    return iterate_f(f, steady_n, x, control_param, store_vals=True, **kwargs)


def get_bifurcation_data(
    f: Callable,
    steady_n: int,
    transient_n: int,
    x_init: float,
    control_param_vals: List[float],
    **kwargs
) -> {str: List[float], str: List[float]}:
    cp_vals = []
    state_vals = []
    for cp in control_param_vals:
        steady_vals = f_steady_vals(f, steady_n, transient_n, x_init, cp, **kwargs)
        cp_vals += [cp] * steady_n
        state_vals += steady_vals            
            
    return {"cp_vals": cp_vals, "state_vals": state_vals}


def get_bifurcation_fig(
    title: str,
    f: Callable,
    steady_n: int,
    transient_n: int,
    x_init: float,
    control_param_vals: List[float],
    **kwargs
):
    data = get_bifurcation_data(
        f, steady_n, transient_n, x_init, control_param_vals, **kwargs
    )
    xs = data["cp_vals"]
    ys = data["state_vals"]

    fig, ax = plt.subplots()
    ax.plot(xs, ys, ",k")
    ax.set_title(title)
    ax.set_xlabel("Control Parameter")
    ax.set_ylabel("State")

    return fig


In [ ]:
"""
Single experiment for a given starting 'x_init' and constant control param 'r' over 'n' iterations.
"""
n = 100000
x_init = .45
r = 3.55
ys = iterate_f(logistic_map, n, x_init, r, store_vals=True)

plt.figure()
plt.plot(range(n), ys, ',k')  
plt.title(f'Logistic Map for x_init = {x_init}, r = {r}')
plt.xlabel('Control Param $r$')
plt.ylabel('State')
plt.show()

In [ ]:
# Logistic Map
f = logistic_map
steady_n = 100
transient_n = 1000
x_init = 0.4
control_param_vals = np.linspace(-2, 4, 1000)
title = 'Logistic Map Bifurcation: $x_{n+1}=rx_{n}(1-x_{n})$'
fig = get_bifurcation_fig(title, f, steady_n, transient_n, x_init, control_param_vals)
fig.savefig('figs/bifurcation_plots/logistic_map.png')

In [ ]:
# Sine Map
f = sine_map
steady_n = 100
transient_n = 10000
x_init = 0.4
control_param_vals = np.linspace(-4, 4, 1000)
title = 'Sine Map Bifurcation: $x_{n+1}=r \sin(\pi x_{n})$'
fig = get_bifurcation_fig(title, f, steady_n, transient_n, x_init, control_param_vals)
fig.savefig('figs/bifurcation_plots/sine_map.png')

In [ ]:
# Tent Map
f = tent_map
steady_n = 100
transient_n = 10000
x_init = 0.4
control_param_vals = np.linspace(-2, 2, 1000)
title = 'Tent Map Bifurcation: $x_{n+1}=r(1-2|x_{n}-0.5|)$'
fig = get_bifurcation_fig(title, f, steady_n, transient_n, x_init, control_param_vals)
fig.savefig('figs/bifurcation_plots/tent_map.png')